# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-08-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-08-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-09 05:28:01,33.93911,67.709953,47716,1906,37920,7890.0,Afghanistan,122.573937,3.994467
1,NaN,NaN,NaN,Albania,2020-12-09 05:28:01,41.15330,20.168300,44436,936,22527,20973.0,Albania,1544.096185,2.106400
2,NaN,NaN,NaN,Algeria,2020-12-09 05:28:01,28.03390,1.659600,89416,2539,58146,28731.0,Algeria,203.908491,2.839537
3,NaN,NaN,NaN,Andorra,2020-12-09 05:28:01,42.50630,1.521800,7127,78,6367,682.0,Andorra,9224.098880,1.094430
4,NaN,NaN,NaN,Angola,2020-12-09 05:28:01,-11.20270,17.873900,15729,355,8470,6904.0,Angola,47.857578,2.256978


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46116,46274,46516,46718,46837,46837,47072,47306,47516,47716
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,37625,38182,39014,39719,40501,41302,42148,42988,43683,44436
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,82221,83199,84152,85084,85927,86730,87502,88252,88825,89416


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1774,1795,1822,1841,1846,1846,1864,1874,1900,1906
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,798,810,822,839,852,870,889,905,922,936
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2410,2431,2447,2464,2480,2492,2501,2516,2527,2539


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36716,36831,36946,37218,37260,37260,37393,37685,37879,37920
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,18481,18849,19384,19912,20484,20974,21286,21617,22180,22527
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,53204,53809,54405,54990,55538,56079,56617,57146,57648,58146


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2751,2780,2818,2873,2893,2945,2979,3005,3043,3087
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,8820,8890,9051,9163,9341,9501,9626,9728,9821,9974
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1175,1178,1189,1206,1214,1217,1219,1223,1224,1240


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,42,42,42,42,42,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,98,98,137,137,137,138,138,138,138,141
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,11,11,11,29,29,29,29,29,29,30


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-09 05:28:01,32.539527,-86.644082,3087,42,0,3045.0,"Autauga, Alabama, US",5525.425549,1.360544
686,1075.0,Lamar,Alabama,US,2020-12-09 05:28:01,33.779950,-88.096680,769,8,0,761.0,"Lamar, Alabama, US",5570.445491,1.040312
687,1077.0,Lauderdale,Alabama,US,2020-12-09 05:28:01,34.901719,-87.656247,4722,57,0,4665.0,"Lauderdale, Alabama, US",5092.258085,1.207116


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,15164885,286237,5786915
India,9735850,141360,9215581
Brazil,6674999,178159,5965492


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,15164885,286237,5786915,9091733,2020-12-09 05:28:01,37.936303,-91.379001
India,9735850,141360,9215581,378909,2020-12-09 05:28:01,23.088275,81.806127
Brazil,6674999,178159,5965492,531348,2020-12-09 05:28:01,-12.669522,-48.480493
Russia,2492713,43674,1963864,485175,2020-12-09 05:28:01,54.546312,62.120860
France,2363196,56453,177558,2129185,2020-12-09 05:28:01,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1415396,20205,0
Texas,1346643,23372,0
Florida,1073770,19378,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1415396,20205,0,1395191,2020-12-09 05:28:01,37.843962,-120.728594
Texas,1346643,23372,0,1323271,2020-12-09 05:28:01,31.660643,-98.653069
Florida,1073770,19378,0,1054392,2020-12-09 05:28:01,28.940755,-82.700744
Illinois,804174,14384,0,789790,2020-12-09 05:28:01,39.843618,-89.178330
New York,722464,35118,0,687346,2020-12-09 05:28:01,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,466321,8000,0
Illinois,Cook,335204,7051,0
Florida,Miami-Dade,246915,3926,0
Arizona,Maricopa,236818,4161,0
Texas,Harris,200397,3086,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,466321,8000,0,458321,2020-12-09 05:28:01,34.308284,-118.228241,6037.0
Illinois,Cook,335204,7051,0,328153,2020-12-09 05:28:01,41.841448,-87.816588,17031.0
Florida,Miami-Dade,246915,3926,0,242989,2020-12-09 05:28:01,25.611236,-80.551706,12086.0
Arizona,Maricopa,236818,4161,0,232657,2020-12-09 05:28:01,33.348359,-112.491815,4013.0
Texas,Harris,200397,3086,0,197311,2020-12-09 05:28:01,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,47306,42988,88252,7050,15591,144,1463110,141937,27972,303430,...,1727751,7303,74053,1,104442,1366,98038,2337,17916,10718
2020-12-07,47516,43683,88825,7084,15648,146,1466309,142344,27987,305693,...,1742525,7505,74053,1,104904,1367,99758,2383,17931,10839
2020-12-08,47716,44436,89416,7127,15729,146,1469919,142928,27993,308070,...,1754911,7806,74206,1,105384,1377,101109,2078,17963,10912


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,1874,905,2516,78,354,4,39770,2326,908,3840,...,61342,82,611,0,919,35,828,639,364,291
2020-12-07,1900,922,2527,78,354,4,39888,2344,908,3897,...,61531,83,611,0,924,35,848,649,364,294
2020-12-08,1906,936,2539,78,355,4,40009,2372,908,4002,...,62130,86,611,0,928,35,867,606,364,303


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,37685,21617,57146,6238,8338,133,1294692,116796,25642,252765,...,3736,5062,71315,1,99494,1220,72773,1549,17173,8880
2020-12-07,37879,22180,57648,6293,8353,134,1300696,117649,25646,256811,...,3747,5240,71315,1,99982,1224,74053,1554,17211,8972
2020-12-08,37920,22527,58146,6367,8470,134,1305587,119147,25651,262314,...,3765,5415,71482,1,100476,1224,75257,1382,17268,9062


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2751,2780,2818,2873,2893,2945,2979,3005,3043,3087
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,683,689,701,713,725,734,744,755,759,769
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,4133,4223,4267,4360,4433,4522,4593,4626,4662,4722
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1296,1316,1334,1353,1373,1399,1421,1422,1431,1453
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7785,7814,7872,7925,8015,8071,8131,8171,8218,8298


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,269877,37036,0,364276,170924,1366673,260581,127715,39912,49,...,400594,1322738,215407,5015,1633,255053,177447,54997,441067,36218
2020-12-07,272229,37539,0,365843,172042,1390828,264618,135844,40711,49,...,408730,1331719,217638,5080,1649,258870,184404,56128,443389,36867
2020-12-08,276665,38116,0,378157,174325,1415396,268589,138258,41464,49,...,414749,1346643,219971,5180,1680,262730,187327,57060,448009,37339


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-06        3005    9728    1223  1293   3299     713   1236    7096   
2020-12-07        3043    9821    1224  1299   3324     714   1244    7165   
2020-12-08        3087    9974    1240  1317   3426     719   1257    7300   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-06         1906     1093  ...    1472    358     2161      496   
2020-12-07         1915     1110  ...    1498    362     2198      508   
2020-12-08         1945     1125  ...    1521    369     2213      516   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-06           2112  1785  1198          0      546    420  
2020-12-07           2176  1810  1221          0      556    422  
2020-12-08           2200  1822  1237          0      577    425  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,3889,143,0,6950,2660,19928,3356,5146,793,0,...,4943,23137,939,79,23,4200,2925,838,3952,266
2020-12-07,3892,146,0,6950,2713,20015,3358,5224,793,0,...,5009,23186,949,81,23,4208,2941,841,3973,289
2020-12-08,3942,146,0,6973,2752,20205,3372,5242,803,0,...,5109,23372,972,85,23,4260,2967,870,4054,289


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-06          42     138      29   38     46      20     42     127   
2020-12-07          42     138      29   39     46      20     42     127   
2020-12-08          42     141      30   39     46      20     42     127   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-06           54       24  ...       6      8       11        2   
2020-12-07           54       24  ...       7      8       12        3   
2020-12-08           54       24  ...       7      8       12        3   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-06             10     2     5          9        8      2  
2020-12-07             11     2     6          9       10      2  
2020-12-08             11     2     6          9       10      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.004971,0.019930,0.008571,0.006424,0.003540,0.000000,0.002245,0.006938,0.000250,0.009116,...,0.010157,0.048528,0.004095,0.0,0.002544,0.000733,0.020188,0.014323,0.001006,0.009513
2020-12-07,0.004439,0.016167,0.006493,0.004823,0.003656,0.013889,0.002186,0.002867,0.000536,0.007458,...,0.008551,0.027660,0.000000,0.0,0.004424,0.000732,0.017544,0.019683,0.000837,0.011289
2020-12-08,0.004209,0.017238,0.006654,0.006070,0.005176,0.000000,0.002462,0.004103,0.000214,0.007776,...,0.007108,0.040107,0.002066,0.0,0.004576,0.007315,0.013543,-0.127990,0.001785,0.006735


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.005365,0.017998,0.005998,0.0,0.000000,0.0,0.003482,0.007362,0.0,0.022092,...,0.003780,0.012346,0.0,NaN,0.003275,0.0,0.019704,0.009479,0.0,0.000000
2020-12-07,0.013874,0.018785,0.004372,0.0,0.000000,0.0,0.002967,0.007739,0.0,0.014844,...,0.003081,0.012195,0.0,NaN,0.005441,0.0,0.024155,0.015649,0.0,0.010309
2020-12-08,0.003158,0.015184,0.004749,0.0,0.002825,0.0,0.003033,0.011945,0.0,0.026944,...,0.009735,0.036145,0.0,NaN,0.004329,0.0,0.022406,-0.066256,0.0,0.030612


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.007809,0.015550,0.009343,0.010857,0.000360,0.000000,0.004583,0.006290,0.000117,0.013326,...,0.023562,0.023453,0.004408,0.0,0.006220,0.000000,0.021576,0.001293,0.000291,0.004071
2020-12-07,0.005148,0.026044,0.008785,0.008817,0.001799,0.007519,0.004637,0.007303,0.000156,0.016007,...,0.002944,0.035164,0.000000,0.0,0.004905,0.003279,0.017589,0.003228,0.002213,0.010360
2020-12-08,0.001082,0.015645,0.008639,0.011759,0.014007,0.000000,0.003760,0.012733,0.000195,0.021428,...,0.004804,0.033397,0.002342,0.0,0.004941,0.000000,0.016259,-0.110682,0.003312,0.010031


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,0.008550,0.021091,NaN,0.014979,0.009104,0.021475,0.012567,0.00000,0.020872,0.0,...,0.007728,0.008459,0.012042,0.024724,0.000000,0.015448,0.009409,0.026600,0.007191,0.009814
2020-12-07,0.008715,0.013581,NaN,0.004302,0.006541,0.017674,0.015492,0.06365,0.020019,0.0,...,0.020310,0.006790,0.010357,0.012961,0.009798,0.014966,0.039206,0.020565,0.005265,0.017919
2020-12-08,0.016295,0.015371,NaN,0.033659,0.013270,0.017664,0.015007,0.01777,0.018496,0.0,...,0.014726,0.011207,0.010720,0.019685,0.018799,0.014911,0.015851,0.016605,0.010420,0.012803


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-06      0.008728  0.010596  0.003281  0.007794  0.005486  0.000000   
2020-12-07      0.012646  0.009560  0.000818  0.004640  0.007578  0.001403   
2020-12-08      0.014459  0.015579  0.013072  0.013857  0.030686  0.007003   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-06      0.008980  0.009819  0.002630  0.002752  ...  0.015873   
2020-12-07      0.006472  0.009724  0.004722  0.015554  ...  0.017663   
2020-12-08      0.010450  0.018842  0.015666  0.013514  ...  0.015354   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-06      0.019943  0.003716  0.006085   0.006673  0.026452  0.009267   
2020-12-07      0.011173  0.017122  0.024194   0.030303  0.014006  0.019199   
2020-12-08      0.019337  0.006824  0.015748   0.011029  0.006630  0.013104   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-06            NaN  0.050000  0.002387  
2020-12-07            NaN  0.018315  0.004762  
2020-12-08            NaN  0.037770  0.007109  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,0.003095,0.000000,NaN,0.003610,0.015267,0.002919,-0.000298,0.000000,0.014066,NaN,...,0.007747,0.003557,0.000000,0.000000,0.0,0.000715,0.000000,0.010856,0.004575,0.000000
2020-12-07,0.000771,0.020979,NaN,0.000000,0.019925,0.004366,0.000596,0.015157,0.000000,NaN,...,0.013352,0.002118,0.010650,0.025316,0.0,0.001905,0.005470,0.003580,0.005314,0.086466
2020-12-08,0.012847,0.000000,NaN,0.003309,0.014375,0.009493,0.004169,0.003446,0.012610,NaN,...,0.019964,0.008022,0.024236,0.049383,0.0,0.012357,0.008841,0.034483,0.020388,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga   Baldwin   Barbour      Bibb Blount Bullock Butler   
2020-12-06         0.0  0.000000  0.000000  0.000000    0.0     0.0    0.0   
2020-12-07         0.0  0.000000  0.000000  0.026316    0.0     0.0    0.0   
2020-12-08         0.0  0.021739  0.034483  0.000000    0.0     0.0    0.0   

Province_State                            ...   Wyoming                   \
Admin2         Calhoun Chambers Cherokee  ...      Park Platte  Sheridan   
2020-12-06         0.0      0.0      0.0  ...  0.000000    0.0  0.000000   
2020-12-07         0.0      0.0      0.0  ...  0.166667    0.0  0.090909   
2020-12-08         0.0      0.0      0.0  ...  0.000000    0.0  0.000000   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-06          0.0        0.0   0.0   0.0        0.0     0.00    0.0  
2020-12-07          0.5        0.1   0.0   0.2        0.0     0.25    0.0  
2020-12-08          0.0        0.0   0.0   0.0        0.0     0.00    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.004179,0.019964,0.009017,0.006966,0.004010,0.000556,0.003262,0.007871,0.000289,0.010745,...,0.009724,0.042571,0.002780,0.0,0.002902,0.001491,0.019808,0.013854,0.002182,0.010136
2020-12-07,0.004309,0.018066,0.007755,0.005894,0.003833,0.007223,0.002724,0.005369,0.000413,0.009102,...,0.009138,0.035116,0.001390,0.0,0.003663,0.001111,0.018676,0.016769,0.001510,0.010713
2020-12-08,0.004259,0.017652,0.007204,0.005982,0.004505,0.003611,0.002593,0.004736,0.000313,0.008439,...,0.008123,0.037611,0.001728,0.0,0.004119,0.004213,0.016110,-0.055611,0.001647,0.008724


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.006014,0.019197,0.005363,0.004069,0.001500,3.973643e-08,0.003746,0.010080,0.000004,0.026273,...,0.005583,0.010496,0.000056,NaN,0.003546,1.003943e-30,0.019973,0.008182,0.002451,0.010111
2020-12-07,0.009944,0.018991,0.004867,0.002035,0.000750,1.986821e-08,0.003356,0.008910,0.000002,0.020558,...,0.004332,0.011346,0.000028,NaN,0.004493,5.019715e-31,0.022064,0.011916,0.001226,0.010210
2020-12-08,0.006551,0.017088,0.004808,0.001017,0.001788,9.934107e-09,0.003195,0.010427,0.000001,0.023751,...,0.007034,0.023745,0.000014,NaN,0.004411,2.509858e-31,0.022235,-0.027170,0.000613,0.020411


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-06,0.005198,0.017888,0.009603,0.009854,0.004008,0.000727,0.004944,0.009090,0.000185,0.018318,...,0.013889,0.024258,0.002959,0.0,0.004132,0.001947,0.020190,0.003981,0.000994,0.005036
2020-12-07,0.005173,0.021966,0.009194,0.009335,0.002904,0.004123,0.004791,0.008197,0.000171,0.017163,...,0.008417,0.029711,0.001479,0.0,0.004518,0.002613,0.018890,0.003605,0.001604,0.007698
2020-12-08,0.003128,0.018805,0.008916,0.010547,0.008455,0.002061,0.004275,0.010465,0.000183,0.019295,...,0.006610,0.031554,0.001911,0.0,0.004730,0.001306,0.017574,-0.053539,0.002458,0.008865


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,0.011022,0.022596,NaN,0.016323,0.011891,0.020245,0.016550,0.005233,0.020237,5.742133e-81,...,0.009877,0.009453,0.014456,0.027190,0.005874,0.014157,0.009994,0.025552,0.010047,0.010676
2020-12-07,0.009869,0.018089,NaN,0.010312,0.009216,0.018960,0.016021,0.034441,0.020128,2.871067e-81,...,0.015093,0.008121,0.012407,0.020075,0.007836,0.014561,0.024600,0.023058,0.007656,0.014298
2020-12-08,0.013082,0.016730,NaN,0.021986,0.011243,0.018312,0.015514,0.026106,0.019312,1.435533e-81,...,0.014910,0.009664,0.011563,0.019880,0.013318,0.014736,0.020226,0.019832,0.009038,0.013550


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-06      0.010886  0.012757  0.003400  0.010027  0.011606  0.001443   
2020-12-07      0.011766  0.011159  0.002109  0.007333  0.009592  0.001423   
2020-12-08      0.013113  0.013369  0.007590  0.010595  0.020139  0.004213   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-06      0.009402  0.012553  0.005268  0.006157  ...  0.016930   
2020-12-07      0.007937  0.011138  0.004995  0.010855  ...  0.017296   
2020-12-08      0.009194  0.014990  0.010330  0.012184  ...  0.016325   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-06      0.013548  0.008410  0.011951   0.013205  0.019148  0.013021   
2020-12-07      0.012361  0.012766  0.018072   0.021754  0.016577  0.016110   
2020-12-08      0.015849  0.009795  0.016910   0.016392  0.011603  0.014607   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-06           -1.0  0.035205  0.002778  
2020-12-07           -1.0  0.026760  0.003770  
2020-12-08           -1.0  0.032265  0.005439  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-06,0.008372,0.017588,NaN,0.005577,0.013593,0.005104,0.005793,0.001583,0.007758,NaN,...,0.009659,0.006875,0.006646,0.012500,8.063404e-12,0.003787,0.003059,0.018758,0.013146,0.008830
2020-12-07,0.004572,0.019283,NaN,0.002789,0.016759,0.004735,0.003194,0.008370,0.003879,NaN,...,0.011506,0.004496,0.008648,0.018908,4.031702e-12,0.002846,0.004264,0.011169,0.009230,0.047648
2020-12-08,0.008709,0.009642,NaN,0.003049,0.015567,0.007114,0.003682,0.005908,0.008245,NaN,...,0.015735,0.006259,0.016442,0.034146,2.015851e-12,0.007602,0.006552,0.022826,0.014809,0.023824


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-06      0.000025  0.007171  0.051527  0.035641  0.008062  0.006579   
2020-12-07      0.000012  0.003586  0.025764  0.030978  0.004031  0.003289   
2020-12-08      0.000006  0.012662  0.030123  0.015489  0.002015  0.001645   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2020-12-06      0.000095  0.007238  0.008618  2.034505e-06  ...  0.000031   
2020-12-07      0.000048  0.003619  0.004309  1.017253e-06  ...  0.083349   
2020-12-08      0.000024  0.001809  0.002154  5.086264e-07  ...  0.041674   

Province_State                                                         \
Admin2            Platte  Sheridan  Sublette Sweetwater         Teton   
2020-12-06      0.002285  0.006251  0.062500   0.041669  5.820766e-11   
2020-12-07      0.001143  0.048580  0.281250   0.070834  2.910383e-11   
2020-12-08      0.000571  0.024290  0.140625   0.035417  1.455192e-11   

Province_State                                              
Admin2             Uinta    Unassigned  Washakie    Weston  
2020-12-06      0.015625 -1.982845e-09  0.000035  0.015627  
2020-12-07      0.107813 -9.756857e-10  0.125017  0.007813  
2020-12-08      0.053906 -4.840016e-10  0.062509  0.003906  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201208,AK,36758,NaN,1052220,NaN,totalTestsViral,1088978,159.0,827.0,...,1088978,0,22,332692318da328d0239cf8586792c45a6c837366,0,0,0,0,0,NaN
1,20201208,AL,276665,47658.0,1435083,NaN,totalTestsPeopleViral,1664090,2097.0,27323.0,...,1711748,50,279,c863532e0b12a00cdb0a9e311c4d17f23651b03f,0,0,0,0,0,NaN
2,20201208,AR,174325,23713.0,1632886,NaN,totalTestsViral,1783498,1081.0,9577.0,...,1807211,39,132,7cbb50beebdac1117b347d8cfc4b32f4c8469dc3,0,0,0,0,0,NaN
3,20201208,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,9ac24265171e23f0f70d49818f69c3eb77a43644,0,0,0,0,0,NaN
4,20201208,AZ,378157,13019.0,2042903,NaN,totalTestsPeopleViral,2408041,3157.0,28525.0,...,2421060,23,252,b992db41bf52b50d3edf7db98ebf738405a03bdb,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201208,AK,36758.0,NaN,1052220.0,NaN,totalTestsViral,1088978.0,159.0,827.0,...,1088978,0,22,332692318da328d0239cf8586792c45a6c837366,0,0,0,0,0,NaN
1,20201208,AL,276665.0,47658.0,1435083.0,NaN,totalTestsPeopleViral,1664090.0,2097.0,27323.0,...,1711748,50,279,c863532e0b12a00cdb0a9e311c4d17f23651b03f,0,0,0,0,0,NaN
2,20201208,AR,174325.0,23713.0,1632886.0,NaN,totalTestsViral,1783498.0,1081.0,9577.0,...,1807211,39,132,7cbb50beebdac1117b347d8cfc4b32f4c8469dc3,0,0,0,0,0,NaN
3,20201208,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,9ac24265171e23f0f70d49818f69c3eb77a43644,0,0,0,0,0,NaN
4,20201208,AZ,378157.0,13019.0,2042903.0,NaN,totalTestsPeopleViral,2408041.0,3157.0,28525.0,...,2421060,23,252,b992db41bf52b50d3edf7db98ebf738405a03bdb,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-08,AK,36758,NaN,1052220,NaN,totalTestsViral,1088978,159.0,827.0,NaN,...,1088978,0,22,332692318da328d0239cf8586792c45a6c837366,0,0,0,0,0,NaN
2020-12-08,AL,276665,47658.0,1435083,NaN,totalTestsPeopleViral,1664090,2097.0,27323.0,NaN,...,1711748,50,279,c863532e0b12a00cdb0a9e311c4d17f23651b03f,0,0,0,0,0,NaN
2020-12-08,AR,174325,23713.0,1632886,NaN,totalTestsViral,1783498,1081.0,9577.0,394.0,...,1807211,39,132,7cbb50beebdac1117b347d8cfc4b32f4c8469dc3,0,0,0,0,0,NaN
2020-12-08,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,9ac24265171e23f0f70d49818f69c3eb77a43644,0,0,0,0,0,NaN
2020-12-08,AZ,378157,13019.0,2042903,NaN,totalTestsPeopleViral,2408041,3157.0,28525.0,744.0,...,2421060,23,252,b992db41bf52b50d3edf7db98ebf738405a03bdb,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-08,AK,36758.0,NaN,1052220.0,NaN,totalTestsViral,1088978.0,159.0,827.0,NaN,...,1088978,0,22,332692318da328d0239cf8586792c45a6c837366,0,0,0,0,0,NaN
2020-12-08,AL,276665.0,47658.0,1435083.0,NaN,totalTestsPeopleViral,1664090.0,2097.0,27323.0,NaN,...,1711748,50,279,c863532e0b12a00cdb0a9e311c4d17f23651b03f,0,0,0,0,0,NaN
2020-12-08,AR,174325.0,23713.0,1632886.0,NaN,totalTestsViral,1783498.0,1081.0,9577.0,394.0,...,1807211,39,132,7cbb50beebdac1117b347d8cfc4b32f4c8469dc3,0,0,0,0,0,NaN
2020-12-08,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,9ac24265171e23f0f70d49818f69c3eb77a43644,0,0,0,0,0,NaN
2020-12-08,AZ,378157.0,13019.0,2042903.0,NaN,totalTestsPeopleViral,2408041.0,3157.0,28525.0,744.0,...,2421060,23,252,b992db41bf52b50d3edf7db98ebf738405a03bdb,0,0,0,0,0,NaN
